In [1]:
import pandas as pd #1
import os           #3
import numpy as np

In [2]:
given_data_folder = 'display_data'

file_names = os.listdir(given_data_folder)
file_names.sort()
file_names

['MNCAATourneySeeds.csv',
 'MNCAATourneySlots.csv',
 'MTeams.csv',
 'WNCAATourneySeeds.csv',
 'WNCAATourneySlots.csv',
 'WTeams.csv']

# Predictions

In [3]:
#predictions_file_name = 'M_predictions_embeddings_2023.csv'
predictions_file_name = 'aem_actual_230315_23.28.csv'

In [4]:
file_path = predictions_file_name 
predictions  = pd.read_csv(file_path)
print(len(predictions))

predictions.head(5)

130683


,ID,Pred
0,2023_1101_1102,0.728315
1,2023_1101_1103,0.426748
2,2023_1101_1104,0.177597
3,2023_1101_1105,0.844958
4,2023_1101_1106,0.894379


# Team Names

In [5]:
teams_file_name = 'WTeams.csv'

In [6]:
file_path = given_data_folder + '/' + teams_file_name 
team_names  = pd.read_csv(file_path)
print(len(team_names))

team_names.head(5)

375


,TeamID,TeamName
0,3101,Abilene Chr
1,3102,Air Force
2,3103,Akron
3,3104,Alabama
4,3105,Alabama A&M


## Slots

In [7]:
tourney_slots_file_name = 'WNCAATourneySlots.csv'

In [8]:
file_path = given_data_folder + '/' + tourney_slots_file_name 
tourney_slots  = pd.read_csv(file_path)
print(len(tourney_slots))

tourney_slots.head(5)

1646


,Season,Slot,StrongSeed,WeakSeed
0,1998,R1W1,W01,W16
1,1998,R1W2,W02,W15
2,1998,R1W3,W03,W14
3,1998,R1W4,W04,W13
4,1998,R1W5,W05,W12


In [9]:
tourney_slots = tourney_slots[tourney_slots.Season == 2023]

print(len(tourney_slots))
#tourney_slots.tail(20)
tourney_slots.head(25)
tourney_slots[45:]

67


,Season,Slot,StrongSeed,WeakSeed
1624,2023,R2Z2,R1Z2,R1Z7
1625,2023,R2Z3,R1Z3,R1Z6
1626,2023,R2Z4,R1Z4,R1Z5
1627,2023,R3W1,R2W1,R2W4
1628,2023,R3W2,R2W2,R2W3
1629,2023,R3X1,R2X1,R2X4
1630,2023,R3X2,R2X2,R2X3
1631,2023,R3Y1,R2Y1,R2Y4
1632,2023,R3Y2,R2Y2,R2Y3
1633,2023,R3Z1,R2Z1,R2Z4


## Seeds

In [10]:
tourney_seeds_file_name = 'WNCAATourneySeeds.csv'

In [11]:
file_path = given_data_folder + '/' + tourney_seeds_file_name 
tourney_seeds = pd.read_csv(file_path)
print(len(tourney_seeds))

tourney_seeds.head(5)

1608


,Season,Seed,TeamID
0,1998,W01,3330
1,1998,W02,3163
2,1998,W03,3112
3,1998,W04,3301
4,1998,W05,3272


In [12]:
tourney_seeds = tourney_seeds[tourney_seeds.Season == 2023]

print(len(tourney_seeds))
tourney_seeds.head(5)

68


,Season,Seed,TeamID
1540,2023,W01,3376
1541,2023,W02,3268
1542,2023,W03,3323
1543,2023,W04,3417
1544,2023,W05,3328


# Get tourney slots and seeds as columns

In [13]:
def get_as_columns(df):
    # Creates list of the column names from the df
    columns = df.columns.tolist()

    # Creates list of the columns
    df_columns = []
    for expected_column in columns:
        df_columns.append( df[expected_column].tolist() )

    return df_columns

In [14]:
def get_winner(predictions_c, team1, team2):

    for i in range(len(predictions_c[0])):
        teams = predictions_c[0][i].split('_') # 2023_1101_1102

        if team1 == teams[1] and team2 == teams[2]:
            prediction = predictions_c[1][i]
                                 
            if prediction > 0.5: 
                winner = team1
                return winner, prediction
            else:                
                winner = team2
                return winner, (1-prediction)
                
        elif team1 == teams[2] and team2 == teams[1]:
            prediction = predictions_c[1][i]
                                 
            if prediction > 0.5: 
                winner = team2
                return winner, prediction
            else:                
                winner = team1
                return winner, (1-prediction)
            
    #return winner, prediction

In [15]:
def predict_games(game_indicis):
    results = []
    
    for game in game_indicis:
        winning_slot = slots_columns[1][game]

        slot1 = slots_columns[2][game]
        slot2 = slots_columns[3][game]

        for i in range(len(seeds_columns[0])):
            seed = seeds_columns[1][i]

            if seed == slot1: 
                team1 = str(seeds_columns[2][i])
                team1_i = i
            elif seed == slot2: 
                team2 = str(seeds_columns[2][i])
                team2_i = i

        winner, prediction = get_winner(predictions_c, team1, team2)
        
        
        for i in range(len(seeds_columns[0])):
            team_id = seeds_columns[2][i]
            if str(team_id) == str(winner):
                seeds_columns[1][i] = winning_slot
                


        team_name = get_team_name(team_names_c, winner)
        team_region, team_seed = get_team_region_seed(seeds_columns, winner)
        
        out = "{:>4} - {:>6}% | {} {:<7} - {}".format(winning_slot, round(prediction*100, 2), 
                                                             team_seed, team_region, team_name)
        print(out)
        
        results.append([winning_slot, team_name, prediction])
        
    return results

In [16]:
def get_seed_region(seeds_columns):
    '''
    W = East
    X = South
    Y = Midwest
    Z = West
    
    firt char is region
    second two chars is seed
    '''
    conferences_dict = {'W': "East", 'X': "South", 'Y': "Midwest", 'Z': "West"}
    
    conferences = []
    seed_nums   = []

    for i in range(len(seeds_columns[0])):
        base_seed = seeds_columns[1][i]
        
        conference = base_seed[0]
        conferences.append(conferences_dict[conference])
        
        seed_num = base_seed[1:3]
        seed_nums.append(seed_num)
        
    seeds_columns.append(conferences)
    seeds_columns.append(seed_nums)
    

In [17]:
def get_team_region_seed(seeds_columns, team_id):
    # Season, Seed, TeamID, conference, seed_num
    for i in range(len(seeds_columns[0])):
        if str(team_id) == str(seeds_columns[2][i]): 
            return seeds_columns[3][i], seeds_columns[4][i]

In [18]:
def get_team_name(team_names_c, team_id):
    for i in range(len(team_names_c[0])):
        if str(team_id) == str(team_names_c[0][i]): 
            return team_names_c[1][i]

### iterate through to get the matchups
(start with last 4 rows, they are play-in games)

In [19]:
# seeds: Season, Seed, TeamID

In [20]:
seeds_columns = get_as_columns(tourney_seeds)
slots_columns = get_as_columns(tourney_slots)
predictions_c = get_as_columns(predictions)
team_names_c  = get_as_columns(team_names)

get_seed_region(seeds_columns)

In [21]:
slots_indicis = list(range(len(slots_columns[0])))

play_in_games = slots_indicis[63:67]
round_1_games = slots_indicis[ 0:32]
round_2_games = slots_indicis[32:48]
round_3_games = slots_indicis[48:56]
round_4_games = slots_indicis[56:60]
round_5_games = slots_indicis[60:62]
championship  = slots_indicis[62:63]

In [22]:
rounds = [
    play_in_games,
    round_1_games,
    round_2_games,
    round_3_games,
    round_4_games,
    round_5_games,
    championship
]

In [23]:
round_winners = []

for i in range(len(rounds)):
    winners = predict_games(rounds[i])
    print()

 W11 -  57.91% | 11 East    - Mississippi St
 X16 -  65.85% | 16 South   - Sacred Heart
 Y16 -   62.7% | 16 Midwest - Tennessee Tech
 Z11 -  65.25% | 11 West    - Purdue

R1W1 -  76.88% | 01 East    - South Carolina
R1W2 -  82.76% | 02 East    - Maryland
R1W3 -  60.62% | 03 East    - Notre Dame
R1W4 -  54.39% | 04 East    - UCLA
R1W5 -  62.44% | 05 East    - Oklahoma
R1W6 -  60.17% | 06 East    - Creighton
R1W7 -  68.35% | 07 East    - Arizona
R1W8 -  52.54% | 08 East    - South Florida
R1X1 -  76.32% | 01 South   - Stanford
R1X2 -  59.65% | 02 South   - Iowa
R1X3 -  64.94% | 03 South   - Duke
R1X4 -   70.5% | 04 South   - Texas
R1X5 -  71.06% | 12 South   - Drake
R1X6 -  62.45% | 11 South   - MTSU
R1X7 -  56.54% | 10 South   - Georgia
R1X8 -  52.77% | 08 South   - Mississippi
R1Y1 -  79.47% | 01 Midwest - Indiana
R1Y2 -  56.25% | 15 Midwest - Gardner Webb
R1Y3 -  52.94% | 14 Midwest - Hawaii
R1Y4 -  62.28% | 04 Midwest - Villanova
R1Y5 -  80.39% | 12 Midwest - FL Gulf Coast
R1Y6 -  65

In [24]:
'''
W16b ->  W16 -  67.49% | 16 East    TX Southern
X16b ->  X16 -  66.51% | 16 South   TAM C. Christi
Y11a ->  Y11 -  61.54% | 11 Midwest Mississippi St
Z11a ->  Z11 -  55.37% | 11 West    Arizona St

 W01 -> R1W1 -  89.46% | 01 East    Purdue
 W02 -> R1W2 -  56.56% | 02 East    Marquette
 W03 -> R1W3 -  69.44% | 03 East    Kansas St
 W04 -> R1W4 -  78.58% | 04 East    Tennessee
 W05 -> R1W5 -   79.0% | 05 East    Duke
 W06 -> R1W6 -  58.08% | 06 East    Kentucky
 W07 -> R1W7 -  60.31% | 07 East    Michigan St
 W09 -> R1W8 -   70.6% | 09 East    FL Atlantic
 X01 -> R1X1 -  89.33% | 01 South   Alabama
 X02 -> R1X2 -  86.36% | 02 South   Arizona
 X03 -> R1X3 -  74.48% | 03 South   Baylor
 X13 -> R1X4 -  55.67% | 13 South   Furman
 X12 -> R1X5 -  77.34% | 12 South   Col Charleston
 X06 -> R1X6 -  58.81% | 06 South   Creighton
 X10 -> R1X7 -  54.95% | 10 South   Utah St
 X08 -> R1X8 -  59.06% | 08 South   Maryland
 Y01 -> R1Y1 -  93.46% | 01 Midwest Houston
 Y15 -> R1Y2 -  78.61% | 15 Midwest Colgate
 Y14 -> R1Y3 -  80.74% | 14 Midwest Kennesaw
 Y04 -> R1Y4 -  50.62% | 04 Midwest Indiana
 Y12 -> R1Y5 -  64.77% | 12 Midwest Drake
 Y11 -> R1Y6 -  56.01% | 11 Midwest Mississippi St
 Y10 -> R1Y7 -  69.59% | 10 Midwest Penn St
 Y09 -> R1Y8 -  54.06% | 09 Midwest Auburn
 Z16 -> R1Z1 -  95.99% | 16 West    Howard
 Z02 -> R1Z2 -  89.87% | 02 West    UCLA
 Z03 -> R1Z3 -  82.95% | 03 West    Gonzaga
 Z04 -> R1Z4 -   77.7% | 04 West    Connecticut
 Z05 -> R1Z5 -   62.4% | 05 West    St Mary's CA
 Z11 -> R1Z6 -  54.06% | 11 West    Arizona St
 Z10 -> R1Z7 -  59.12% | 10 West    Boise St
 Z08 -> R1Z8 -  68.25% | 08 West    Arkansas

R1W8 -> R2W1 -  76.17% | 09 East    FL Atlantic
R1W7 -> R2W2 -  56.52% | 07 East    Michigan St
R1W6 -> R2W3 -  64.43% | 06 East    Kentucky
R1W4 -> R2W4 -  50.95% | 04 East    Tennessee
R1X1 -> R2X1 -  59.47% | 01 South   Alabama
R1X2 -> R2X2 -  70.73% | 02 South   Arizona
R1X6 -> R2X3 -  56.55% | 06 South   Creighton
R1X5 -> R2X4 -   67.1% | 12 South   Col Charleston
R1Y8 -> R2Y1 -  69.12% | 09 Midwest Auburn
R1Y7 -> R2Y2 -  61.25% | 10 Midwest Penn St
R1Y6 -> R2Y3 -   82.2% | 11 Midwest Mississippi St
R1Y5 -> R2Y4 -  51.49% | 12 Midwest Drake
R1Z1 -> R2Z1 -  95.69% | 16 West    Howard
R1Z7 -> R2Z2 -  76.58% | 10 West    Boise St
R1Z6 -> R2Z3 -   74.8% | 11 West    Arizona St
R1Z4 -> R2Z4 -  54.54% | 04 West    Connecticut

R2W4 -> R3W1 -   77.7% | 04 East    Tennessee
R2W2 -> R3W2 -  57.97% | 07 East    Michigan St
R2X1 -> R3X1 -  76.36% | 01 South   Alabama
R2X2 -> R3X2 -  55.71% | 02 South   Arizona
R2Y1 -> R3Y1 -  58.19% | 09 Midwest Auburn
R2Y2 -> R3Y2 -  57.67% | 10 Midwest Penn St
R2Z1 -> R3Z1 -  95.25% | 16 West    Howard
R2Z3 -> R3Z2 -  51.43% | 11 West    Arizona St

R3W2 -> R4W1 -  55.55% | 07 East    Michigan St
R3X1 -> R4X1 -  50.42% | 01 South   Alabama
R3Y1 -> R4Y1 -  66.08% | 09 Midwest Auburn
R3Z1 -> R4Z1 -  87.52% | 16 West    Howard

R4W1 -> R5WX -  59.83% | 07 East    Michigan St
R4Y1 -> R5YZ -   91.9% | 09 Midwest Auburn

R5WX -> R6CH -  50.98% | 07 East    Michigan St

''';